In [ ]:
# Stack Overflow 2018 Developer Survey Analysis
## Salary Prediction & Skill Clustering in the Global Software Market

# **Dataset:** Stack Overflow 2018 Developer Survey  
# **Objective:** Examine salary trends and skill demand using regression-based salary prediction and K-Means clustering  
# **Tools:** pandas, numpy, matplotlib, seaborn, scikit-learn

In [ ]:
# ========================
# 1. IMPORT LIBRARIES
# ========================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.cluster import KMeans
from sklearn.metrics import mean_squared_error, r2_score, silhouette_score
import warnings
warnings.filterwarnings('ignore')

# Set plot style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 12

print("✅ All libraries imported successfully!")

---
# 2. Data Loading & Initial Exploration

In [ ]:
# ========================
# 2. LOAD DATA (Fetched from Kaggle)
# ========================
import os
import shutil
import kagglehub

DATA_DIR = os.path.join(os.getcwd(), 'data')
os.makedirs(DATA_DIR, exist_ok=True)

public_csv = os.path.join(DATA_DIR, 'survey_results_public.csv')
schema_csv = os.path.join(DATA_DIR, 'survey_results_schema.csv')

# Download from Kaggle if not already present
if not os.path.exists(public_csv):
    print("⬇️  Downloading Stack Overflow 2018 Developer Survey from Kaggle...")
    path = kagglehub.dataset_download("stackoverflow/stack-overflow-2018-developer-survey")
    print(f"   Downloaded to: {path}")
    # Copy CSVs into our data/ folder
    for f in os.listdir(path):
        if f.endswith('.csv'):
            shutil.copy2(os.path.join(path, f), DATA_DIR)
    print(f"   Copied CSVs to: {DATA_DIR}")
else:
    print("✅ Dataset already exists in data/ — skipping download")

df = pd.read_csv(public_csv)
schema = pd.read_csv(schema_csv)

print(f"\n✅ Data loaded from: {DATA_DIR}")
print(f"Dataset Shape: {df.shape}")
print(f"Total Respondents: {df.shape[0]:,}")
print(f"Total Features: {df.shape[1]}")
print("\n--- First 5 Rows ---")
df.head()

In [ ]:
# Basic info about the dataset
print("--- Data Types & Non-Null Counts ---")
print(df.info())
print("\n--- Statistical Summary ---")
df.describe()

---
# 3. Data Cleaning & Preprocessing

### Cleaning Strategy & Rationale

| Data Issue | Strategy | Why |
|---|---|---|
| **Missing salary (`ConvertedSalary`)** | **Delete** rows with null/zero salary | Salary is our prediction target — imputing it (mean/median) would fabricate the very value we're trying to predict, introducing circular bias into both regression and clustering. |
| **Salary outliers** | **Delete** rows outside 1st–99th percentile | Extreme values (e.g., $1 or $10M/year) are likely data-entry errors. They skew the mean, distort regression coefficients, and pull K-Means centroids. |
| **Missing categorical features** (Country, DevType, Gender, etc.) | **Fill with `'Unknown'`** (a new category) | Deleting every row with any missing categorical would slash the dataset (many columns have 10-30% missing). `'Unknown'` preserves the row's other valid data. For multi-value fields like `LanguageWorkedWith`, `'Unknown'` maps to 0 skills — neutral and interpretable. |
| **Missing ordinal features** (YearsCoding, Satisfaction, Education) | **Fill with neutral midpoint** (0 for years, 4 for satisfaction, 3 for education) | These are mapped to manual numeric scales. A neutral midpoint avoids pulling averages in either direction. This is more semantically meaningful than a blind statistical median for categorical/ordinal data. |

> **Why no median imputation?** For the target variable, deletion is the only honest choice. For features, category-aware filling (`'Unknown'` / neutral value) is more interpretable than a blind statistical median — especially since most features here are categorical or ordinal, not continuous.

In [ ]:
# ========================
# 3a. SELECT RELEVANT COLUMNS & MISSING VALUE ANALYSIS
# ========================

# Select relevant columns for analysis
cols_of_interest = [
    'Country', 'Employment', 'FormalEducation', 'UndergradMajor',
    'CompanySize', 'DevType', 'YearsCoding', 'YearsCodingProf',
    'JobSatisfaction', 'CareerSatisfaction', 'ConvertedSalary',
    'LanguageWorkedWith', 'FrameworkWorkedWith', 'DatabaseWorkedWith',
    'PlatformWorkedWith', 'Gender', 'Age', 'Exercise', 'HoursComputer'
]

df_clean = df[cols_of_interest].copy()
print(f"Selected {len(cols_of_interest)} columns for analysis")

# --- Missing Value Analysis ---
missing = df_clean.isnull().sum()
missing_pct = (missing / len(df_clean) * 100).round(2)
missing_df = pd.DataFrame({'Missing Count': missing, 'Missing %': missing_pct})
missing_df = missing_df[missing_df['Missing Count'] > 0].sort_values('Missing %', ascending=False)
print("\n--- Missing Values (Top Columns) ---")
print(missing_df)

# Visualize missing data
fig, ax = plt.subplots(figsize=(12, 6))
missing_df['Missing %'].plot(kind='barh', color='coral', edgecolor='black', ax=ax)
ax.set_title('Missing Data Percentage by Feature', fontsize=14, fontweight='bold')
ax.set_xlabel('Missing %')
plt.tight_layout()
plt.show()

In [ ]:
# ========================
# 3b. DROP MISSING SALARY & FILTER OUTLIERS
# ========================

# STRATEGY: DELETE rows where salary is missing or zero.
# WHY: ConvertedSalary is our prediction TARGET — imputing it with mean/median
# would fabricate the value we're trying to predict, creating circular bias
# in both the regression model and K-Means clustering.
df_clean = df_clean.dropna(subset=['ConvertedSalary'])
df_clean = df_clean[df_clean['ConvertedSalary'] > 0]

print(f"Rows after dropping missing/zero salary: {len(df_clean):,}")

# STRATEGY: DELETE extreme salary outliers (keep within 1st and 99th percentile).
# WHY: Values like $1/year or $10M/year are likely data-entry errors.
# They skew the mean, distort regression coefficients, and pull K-Means
# cluster centroids away from meaningful positions.
q1 = df_clean['ConvertedSalary'].quantile(0.01)
q99 = df_clean['ConvertedSalary'].quantile(0.99)
df_clean = df_clean[(df_clean['ConvertedSalary'] >= q1) & (df_clean['ConvertedSalary'] <= q99)]
print(f"Rows after removing salary outliers (1st-99th percentile): {len(df_clean):,}")
print(f"Salary range: ${df_clean['ConvertedSalary'].min():,.0f} - ${df_clean['ConvertedSalary'].max():,.0f}")
print(f"Mean Salary: ${df_clean['ConvertedSalary'].mean():,.0f}")
print(f"Median Salary: ${df_clean['ConvertedSalary'].median():,.0f}")

In [ ]:
# ========================
# 3c. ENCODE CATEGORICAL VARIABLES & FILL MISSING VALUES
# ========================

# STRATEGY: FILL missing categorical values with 'Unknown' (a new category).
# WHY: Deleting every row with any missing categorical value would slash the
# dataset dramatically (many columns have 10-30% missing). 'Unknown' preserves
# the row's other valid data. For multi-value fields like LanguageWorkedWith,
# 'Unknown' naturally maps to 0 skills — a neutral, interpretable value.
cat_cols = df_clean.select_dtypes(include='object').columns.tolist()
for col in cat_cols:
    df_clean[col] = df_clean[col].fillna('Unknown')

# STRATEGY: Map ordinal features to numeric values, FILL missing with NEUTRAL MIDPOINTS.
# WHY: These are manual numeric scales (not continuous), so a neutral midpoint
# avoids pulling averages in either direction. This is more semantically
# meaningful than a blind statistical median for categorical/ordinal data.

# Map YearsCoding / YearsCodingProf to numeric midpoints (Unknown → 0 years)
years_map = {
    '0-2 years': 1, '3-5 years': 4, '6-8 years': 7,
    '9-11 years': 10, '12-14 years': 13, '15-17 years': 16,
    '18-20 years': 19, '21-23 years': 22, '24-26 years': 25,
    '27-29 years': 28, '30 or more years': 32, 'Unknown': 0
}
df_clean['YearsCoding_Num'] = df_clean['YearsCoding'].map(years_map).fillna(0)
df_clean['YearsCodingProf_Num'] = df_clean['YearsCodingProf'].map(years_map).fillna(0)

# Map JobSatisfaction to numeric (Unknown → 4, the neutral middle of 1-7 scale)
satisfaction_map = {
    'Extremely dissatisfied': 1, 'Moderately dissatisfied': 2,
    'Slightly dissatisfied': 3, 'Neither satisfied nor dissatisfied': 4,
    'Slightly satisfied': 5, 'Moderately satisfied': 6,
    'Extremely satisfied': 7, 'Unknown': 4
}
df_clean['JobSatisfaction_Num'] = df_clean['JobSatisfaction'].map(satisfaction_map).fillna(4)
df_clean['CareerSatisfaction_Num'] = df_clean['CareerSatisfaction'].map(satisfaction_map).fillna(4)

# Count number of languages, frameworks, databases, platforms known
# ('Unknown' → 0 skills, which is the natural neutral value)
df_clean['Num_Languages'] = df_clean['LanguageWorkedWith'].apply(
    lambda x: len(str(x).split(';')) if x != 'Unknown' else 0
)
df_clean['Num_Frameworks'] = df_clean['FrameworkWorkedWith'].apply(
    lambda x: len(str(x).split(';')) if x != 'Unknown' else 0
)
df_clean['Num_Databases'] = df_clean['DatabaseWorkedWith'].apply(
    lambda x: len(str(x).split(';')) if x != 'Unknown' else 0
)
df_clean['Num_Platforms'] = df_clean['PlatformWorkedWith'].apply(
    lambda x: len(str(x).split(';')) if x != 'Unknown' else 0
)
df_clean['Total_Skills'] = (df_clean['Num_Languages'] + df_clean['Num_Frameworks'] +
                             df_clean['Num_Databases'] + df_clean['Num_Platforms'])

# Label encode Education (Unknown → 3, mid-range neutral value)
edu_order = {
    'I never completed any formal education': 0,
    'Primary/elementary school': 1,
    'Secondary school (e.g. American high school, German Realschule or Gymnasium, etc.)': 2,
    'Some college/university study without earning a degree': 3,
    'Associate degree': 4,
    "Bachelor's degree (BA, BS, B.Eng., etc.)": 5,
    "Master's degree (MA, MS, M.Eng., MBA, etc.)": 6,
    'Professional degree (JD, MD, etc.)': 7,
    'Other doctoral degree (Ph.D, Ed.D., etc.)': 8,
    'Unknown': 3
}
df_clean['Education_Level'] = df_clean['FormalEducation'].map(edu_order).fillna(3)

print(f"✅ Feature engineering complete!")
print(f"Final dataset shape: {df_clean.shape}")
print(f"\nNew numeric features created:")
print("  - YearsCoding_Num, YearsCodingProf_Num")
print("  - JobSatisfaction_Num, CareerSatisfaction_Num")
print("  - Num_Languages, Num_Frameworks, Num_Databases, Num_Platforms")
print("  - Total_Skills, Education_Level")
df_clean.head()

---
# 4. Exploratory Data Analysis (EDA)
Visualizing salary distributions, top countries, skill demand, and correlations.

In [ ]:
# ========================
# 4a. SALARY DISTRIBUTION
# ========================
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Histogram
axes[0].hist(df_clean['ConvertedSalary'], bins=50, color='steelblue', edgecolor='black', alpha=0.7)
axes[0].set_title('Distribution of Annual Salary (USD)', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Converted Salary (USD)')
axes[0].set_ylabel('Frequency')
axes[0].axvline(df_clean['ConvertedSalary'].median(), color='red', linestyle='--', label=f"Median: ${df_clean['ConvertedSalary'].median():,.0f}")
axes[0].legend()

# Box plot
axes[1].boxplot(df_clean['ConvertedSalary'], vert=True, patch_artist=True,
                boxprops=dict(facecolor='lightblue'))
axes[1].set_title('Salary Box Plot', fontsize=14, fontweight='bold')
axes[1].set_ylabel('Converted Salary (USD)')

plt.tight_layout()
plt.show()

In [ ]:
# ========================
# 4b. TOP 15 COUNTRIES BY MEDIAN SALARY
# ========================
country_salary = (df_clean.groupby('Country')['ConvertedSalary']
                  .agg(['median', 'count'])
                  .rename(columns={'median': 'Median_Salary', 'count': 'Respondents'})
                  .query('Respondents >= 50')
                  .sort_values('Median_Salary', ascending=False)
                  .head(15))

fig, ax = plt.subplots(figsize=(14, 7))
bars = ax.barh(country_salary.index[::-1], country_salary['Median_Salary'][::-1],
               color=plt.cm.viridis(np.linspace(0.3, 0.9, 15)), edgecolor='black')
ax.set_title('Top 15 Countries by Median Developer Salary', fontsize=14, fontweight='bold')
ax.set_xlabel('Median Annual Salary (USD)')
for i, (val, count) in enumerate(zip(country_salary['Median_Salary'][::-1], country_salary['Respondents'][::-1])):
    ax.text(val + 1000, i, f'${val:,.0f} (n={count})', va='center', fontsize=9)
plt.tight_layout()
plt.show()

print("\n📊 Insight: The US, Switzerland, and Israel typically lead in developer compensation,")
print("   reflecting high demand-supply gaps in their tech labor markets.")

In [ ]:
# ========================
# 4c. TOP PROGRAMMING LANGUAGES BY POPULARITY
# ========================
# Explode the semicolon-separated languages
all_languages = df_clean['LanguageWorkedWith'].str.split(';').explode()
lang_counts = all_languages.value_counts().head(20)

fig, ax = plt.subplots(figsize=(14, 7))
colors = plt.cm.RdYlGn(np.linspace(0.2, 0.8, 20))
lang_counts[::-1].plot(kind='barh', color=colors, edgecolor='black', ax=ax)
ax.set_title('Top 20 Programming Languages (by Developer Usage)', fontsize=14, fontweight='bold')
ax.set_xlabel('Number of Developers')
plt.tight_layout()
plt.show()

print("\n📊 Insight: JavaScript, HTML/CSS, SQL dominate - reflecting web-centric demand.")
print("   Python's strong position highlights the growing data science & AI market.")

In [ ]:
# ========================
# 4d. SALARY BY EDUCATION LEVEL
# ========================
edu_salary = (df_clean.groupby('FormalEducation')['ConvertedSalary']
              .median()
              .sort_values(ascending=True))
# Keep only meaningful education levels
edu_salary = edu_salary[edu_salary.index != 'Unknown']

fig, ax = plt.subplots(figsize=(14, 6))
edu_salary.plot(kind='barh', color='teal', edgecolor='black', ax=ax)
ax.set_title('Median Salary by Education Level', fontsize=14, fontweight='bold')
ax.set_xlabel('Median Annual Salary (USD)')
ax.set_ylabel('')
plt.tight_layout()
plt.show()

print("\n📊 Insight: Higher education correlates with higher salaries, but the gap between")
print("   Bachelor's and Master's is relatively small — signaling experience matters more.")

In [ ]:
# ========================
# 4e. SALARY vs EXPERIENCE (Scatter + Trend)
# ========================
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Salary vs Years Coding
axes[0].scatter(df_clean['YearsCoding_Num'], df_clean['ConvertedSalary'],
                alpha=0.1, s=10, color='steelblue')
# Add trend line
z = np.polyfit(df_clean['YearsCoding_Num'], df_clean['ConvertedSalary'], 1)
p = np.poly1d(z)
x_line = np.linspace(0, 35, 100)
axes[0].plot(x_line, p(x_line), 'r-', linewidth=2, label=f'Trend (slope={z[0]:,.0f})')
axes[0].set_title('Salary vs Years of Coding Experience', fontsize=13, fontweight='bold')
axes[0].set_xlabel('Years Coding')
axes[0].set_ylabel('Annual Salary (USD)')
axes[0].legend()

# Salary vs Total Skills
axes[1].scatter(df_clean['Total_Skills'], df_clean['ConvertedSalary'],
                alpha=0.1, s=10, color='darkorange')
z2 = np.polyfit(df_clean['Total_Skills'], df_clean['ConvertedSalary'], 1)
p2 = np.poly1d(z2)
x_line2 = np.linspace(0, df_clean['Total_Skills'].max(), 100)
axes[1].plot(x_line2, p2(x_line2), 'r-', linewidth=2, label=f'Trend (slope={z2[0]:,.0f})')
axes[1].set_title('Salary vs Total Technical Skills', fontsize=13, fontweight='bold')
axes[1].set_xlabel('Total Skills (Languages + Frameworks + DBs + Platforms)')
axes[1].set_ylabel('Annual Salary (USD)')
axes[1].legend()

plt.tight_layout()
plt.show()

print("\n📊 Insight: Both experience and breadth of skills show positive correlation with salary.")
print("   This supports the human capital theory — investment in skills yields returns.")

In [ ]:
# ========================
# 4f. CORRELATION HEATMAP
# ========================
numeric_cols = ['ConvertedSalary', 'YearsCoding_Num', 'YearsCodingProf_Num',
                'JobSatisfaction_Num', 'CareerSatisfaction_Num', 'Education_Level',
                'Num_Languages', 'Num_Frameworks', 'Num_Databases', 'Num_Platforms',
                'Total_Skills']

corr_matrix = df_clean[numeric_cols].corr()

fig, ax = plt.subplots(figsize=(12, 10))
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
sns.heatmap(corr_matrix, mask=mask, annot=True, fmt='.2f', cmap='coolwarm',
            center=0, square=True, linewidths=0.5, ax=ax,
            cbar_kws={'label': 'Correlation Coefficient'})
ax.set_title('Correlation Heatmap of Numeric Features', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("\n📊 Insight: Experience (YearsCodingProf) shows the strongest correlation with salary.")
print("   Individual skill counts are moderately correlated with each other (multi-skilled devs).")

---
# 5. K-Means Clustering — Developer Skill Segmentation
Clustering developers based on their skill profiles to identify distinct market segments.

In [ ]:
# ========================
# 5a. PREPARE FEATURES FOR CLUSTERING
# ========================
cluster_features = ['YearsCoding_Num', 'YearsCodingProf_Num', 'Education_Level',
                    'Num_Languages', 'Num_Frameworks', 'Num_Databases',
                    'Num_Platforms', 'Total_Skills', 'ConvertedSalary']

df_cluster = df_clean[cluster_features].dropna()

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df_cluster)

print(f"Clustering dataset shape: {df_cluster.shape}")
print(f"Features used: {cluster_features}")

In [ ]:
# ========================
# 5b. ELBOW METHOD — FIND OPTIMAL K
# ========================
inertias = []
silhouette_scores = []
K_range = range(2, 11)

for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10, max_iter=300)
    labels = kmeans.fit_predict(X_scaled)
    inertias.append(kmeans.inertia_)
    silhouette_scores.append(silhouette_score(X_scaled, labels, sample_size=5000, random_state=42))

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Elbow curve
axes[0].plot(K_range, inertias, 'bo-', linewidth=2, markersize=8)
axes[0].set_title('Elbow Method — Optimal K', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Number of Clusters (K)')
axes[0].set_ylabel('Inertia (Within-Cluster Sum of Squares)')
axes[0].grid(True, alpha=0.3)

# Silhouette scores
axes[1].plot(K_range, silhouette_scores, 'rs-', linewidth=2, markersize=8)
axes[1].set_title('Silhouette Score vs K', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Number of Clusters (K)')
axes[1].set_ylabel('Silhouette Score')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

best_k = K_range[np.argmax(silhouette_scores)]
print(f"\n✅ Best K by Silhouette Score: {best_k}")
print(f"   Silhouette Scores: {dict(zip(K_range, [round(s, 4) for s in silhouette_scores]))}")

In [ ]:
# ========================
# 5c. FIT K-MEANS WITH OPTIMAL K (using K=4 as a balanced choice)
# ========================
optimal_k = 4  # Using 4 clusters for clear business segmentation
kmeans_final = KMeans(n_clusters=optimal_k, random_state=42, n_init=10, max_iter=300)
df_cluster['Cluster'] = kmeans_final.fit_predict(X_scaled)

# Cluster summary statistics
cluster_summary = df_cluster.groupby('Cluster').agg({
    'ConvertedSalary': ['mean', 'median', 'count'],
    'YearsCoding_Num': 'mean',
    'YearsCodingProf_Num': 'mean',
    'Total_Skills': 'mean',
    'Education_Level': 'mean',
    'Num_Languages': 'mean'
}).round(1)

cluster_summary.columns = ['Avg Salary', 'Median Salary', 'Count',
                            'Avg YearsCoding', 'Avg YearsProf',
                            'Avg TotalSkills', 'Avg Education', 'Avg Languages']

# Assign meaningful labels based on characteristics
print("=" * 80)
print("K-MEANS CLUSTER PROFILES")
print("=" * 80)
print(cluster_summary.to_string())
print("\n")

# Rename clusters based on salary and experience
cluster_labels = cluster_summary.sort_values('Median Salary')
for i, (idx, row) in enumerate(cluster_labels.iterrows()):
    labels = ['🟢 Entry-Level / Junior', '🔵 Mid-Level / Generalist',
              '🟡 Senior / Specialist', '🔴 Expert / High-Earner']
    print(f"Cluster {idx}: {labels[i]}")
    print(f"  Median Salary: ${row['Median Salary']:,.0f} | Avg Experience: {row['Avg YearsProf']:.1f} yrs | Avg Skills: {row['Avg TotalSkills']:.1f}")
    print()

In [ ]:
# ========================
# 5d. VISUALIZE CLUSTERS
# ========================
fig, axes = plt.subplots(1, 3, figsize=(20, 6))

colors = ['#2ecc71', '#3498db', '#f1c40f', '#e74c3c']

# Plot 1: Salary vs Experience by Cluster
for c in range(optimal_k):
    mask = df_cluster['Cluster'] == c
    axes[0].scatter(df_cluster.loc[mask, 'YearsCodingProf_Num'],
                    df_cluster.loc[mask, 'ConvertedSalary'],
                    alpha=0.3, s=15, color=colors[c], label=f'Cluster {c}')
axes[0].set_title('Clusters: Salary vs Professional Experience', fontsize=13, fontweight='bold')
axes[0].set_xlabel('Years Coding Professionally')
axes[0].set_ylabel('Annual Salary (USD)')
axes[0].legend()

# Plot 2: Salary vs Total Skills by Cluster
for c in range(optimal_k):
    mask = df_cluster['Cluster'] == c
    axes[1].scatter(df_cluster.loc[mask, 'Total_Skills'],
                    df_cluster.loc[mask, 'ConvertedSalary'],
                    alpha=0.3, s=15, color=colors[c], label=f'Cluster {c}')
axes[1].set_title('Clusters: Salary vs Total Skills', fontsize=13, fontweight='bold')
axes[1].set_xlabel('Total Technical Skills')
axes[1].set_ylabel('Annual Salary (USD)')
axes[1].legend()

# Plot 3: Cluster size distribution
cluster_counts = df_cluster['Cluster'].value_counts().sort_index()
axes[2].bar(cluster_counts.index, cluster_counts.values, color=colors, edgecolor='black')
axes[2].set_title('Cluster Size Distribution', fontsize=13, fontweight='bold')
axes[2].set_xlabel('Cluster')
axes[2].set_ylabel('Number of Developers')
for i, v in enumerate(cluster_counts.values):
    axes[2].text(i, v + 100, f'{v:,}', ha='center', fontweight='bold')

plt.tight_layout()
plt.show()

print("\n📊 Business Insight: The clusters reveal distinct labor market segments —")
print("   from entry-level generalists to highly-specialized senior developers.")
print("   Companies can use these segments for targeted recruitment & compensation benchmarking.")

---
# 6. Linear Regression — Salary Prediction Model
Building a regression model to predict developer salary based on experience, skills, and education.

In [ ]:
# ========================
# 6a. PREPARE FEATURES FOR REGRESSION
# ========================
feature_cols = ['YearsCoding_Num', 'YearsCodingProf_Num', 'Education_Level',
                'Num_Languages', 'Num_Frameworks', 'Num_Databases',
                'Num_Platforms', 'Total_Skills', 'JobSatisfaction_Num',
                'CareerSatisfaction_Num']

target = 'ConvertedSalary'

# Drop NaN
df_reg = df_clean[feature_cols + [target]].dropna()

X = df_reg[feature_cols]
y = df_reg[target]

# Train-Test Split (80-20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Training set: {X_train.shape[0]:,} samples")
print(f"Test set:     {X_test.shape[0]:,} samples")
print(f"Features:     {X_train.shape[1]}")

In [ ]:
# ========================
# 6b. TRAIN LINEAR REGRESSION MODEL
# ========================
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

# Predictions
y_pred_train = lr_model.predict(X_train)
y_pred_test = lr_model.predict(X_test)

# Evaluation Metrics
train_r2 = r2_score(y_train, y_pred_train)
test_r2 = r2_score(y_test, y_pred_test)
train_rmse = np.sqrt(mean_squared_error(y_train, y_pred_train))
test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))

print("=" * 60)
print("LINEAR REGRESSION MODEL PERFORMANCE")
print("=" * 60)
print(f"{'Metric':<25} {'Train':>15} {'Test':>15}")
print("-" * 60)
print(f"{'R² Score':<25} {train_r2:>15.4f} {test_r2:>15.4f}")
print(f"{'RMSE (USD)':<25} {train_rmse:>15,.0f} {test_rmse:>15,.0f}")
print(f"{'Mean Salary (USD)':<25} {y_train.mean():>15,.0f} {y_test.mean():>15,.0f}")
print("=" * 60)

In [ ]:
# ========================
# 6c. FEATURE IMPORTANCE (Regression Coefficients)
# ========================
coef_df = pd.DataFrame({
    'Feature': feature_cols,
    'Coefficient': lr_model.coef_
}).sort_values('Coefficient', ascending=True)

fig, ax = plt.subplots(figsize=(12, 6))
colors = ['#e74c3c' if c < 0 else '#2ecc71' for c in coef_df['Coefficient']]
ax.barh(coef_df['Feature'], coef_df['Coefficient'], color=colors, edgecolor='black')
ax.set_title('Linear Regression — Feature Coefficients (Impact on Salary)', fontsize=14, fontweight='bold')
ax.set_xlabel('Coefficient Value (USD impact per unit increase)')
ax.axvline(x=0, color='black', linewidth=0.8)
plt.tight_layout()
plt.show()

print("\n📊 Interpretation:")
for _, row in coef_df.iterrows():
    direction = "increases" if row['Coefficient'] > 0 else "decreases"
    print(f"  • 1 unit increase in {row['Feature']} {direction} salary by ~${abs(row['Coefficient']):,.0f}")
print(f"\n  Intercept (base salary): ${lr_model.intercept_:,.0f}")

In [ ]:
# ========================
# 6d. ACTUAL vs PREDICTED — VISUALIZATION
# ========================
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Scatter: Actual vs Predicted
axes[0].scatter(y_test, y_pred_test, alpha=0.2, s=10, color='steelblue')
max_val = max(y_test.max(), y_pred_test.max())
axes[0].plot([0, max_val], [0, max_val], 'r--', linewidth=2, label='Perfect Prediction')
axes[0].set_title('Actual vs Predicted Salary', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Actual Salary (USD)')
axes[0].set_ylabel('Predicted Salary (USD)')
axes[0].legend()

# Residual distribution
residuals = y_test - y_pred_test
axes[1].hist(residuals, bins=50, color='coral', edgecolor='black', alpha=0.7)
axes[1].axvline(x=0, color='black', linewidth=1.5, linestyle='--')
axes[1].set_title('Residual Distribution', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Residual (Actual - Predicted) USD')
axes[1].set_ylabel('Frequency')

plt.tight_layout()
plt.show()

print(f"\nMean Residual: ${residuals.mean():,.0f}")
print(f"Std of Residuals: ${residuals.std():,.0f}")
print("\n📊 Insight: The residuals are approximately normally distributed around 0,")
print("   indicating no systematic bias in predictions.")

---
# 7. Business Interpretation & Economic Analysis
Linking data science findings to real-world economic and financial concepts.

In [ ]:
# ========================
# 7a. SALARY BY DEVELOPER TYPE — DEMAND-SUPPLY ANALYSIS
# ========================
# Explode DevType (semicolon-separated)
df_dev = df_clean[['DevType', 'ConvertedSalary']].copy()
df_dev = df_dev[df_dev['DevType'] != 'Unknown']
df_dev_exploded = df_dev.assign(DevType=df_dev['DevType'].str.split(';')).explode('DevType')

dev_stats = (df_dev_exploded.groupby('DevType')['ConvertedSalary']
             .agg(['median', 'count'])
             .rename(columns={'median': 'Median_Salary', 'count': 'Supply'})
             .query('Supply >= 100')
             .sort_values('Median_Salary', ascending=False))

fig, ax1 = plt.subplots(figsize=(14, 8))

# Plot median salary
color1 = 'steelblue'
bars = ax1.barh(dev_stats.index[::-1], dev_stats['Median_Salary'][::-1],
                color=color1, alpha=0.7, edgecolor='black', label='Median Salary')
ax1.set_xlabel('Median Salary (USD)', color=color1)
ax1.set_title('Developer Roles: Salary vs Supply (Demand-Supply Dynamics)', fontsize=14, fontweight='bold')
ax1.tick_params(axis='x', labelcolor=color1)

# Overlay supply count
ax2 = ax1.twiny()
color2 = 'coral'
ax2.plot(dev_stats['Supply'][::-1], dev_stats.index[::-1], 'D-', color=color2,
         markersize=5, linewidth=1.5, label='Developer Supply')
ax2.set_xlabel('Number of Developers (Supply)', color=color2)
ax2.tick_params(axis='x', labelcolor=color2)

fig.legend(loc='lower right', bbox_to_anchor=(0.95, 0.05))
plt.tight_layout()
plt.show()

print("\n📊 Economic Insight — Demand-Supply Analysis:")
print("   • Roles like Engineering Manager and DevOps command HIGH salary with LOWER supply")
print("     → Classic supply shortage driving up equilibrium wages")
print("   • Full-stack/Web developers have HIGH supply but MODERATE salary")
print("     → Competitive market with more supply meeting demand")
print("   • Data Scientists show RISING salary — reflecting growing demand outpacing supply")

In [ ]:
# ========================
# 7b. SKILL PREMIUM ANALYSIS — ROI ON LEARNING LANGUAGES
# ========================
# For each language, compute median salary of devs who know it vs those who don't
all_langs = df_clean['LanguageWorkedWith'].str.split(';').explode().value_counts()
top_langs = all_langs.head(15).index.tolist()

lang_premiums = []
for lang in top_langs:
    knows = df_clean[df_clean['LanguageWorkedWith'].str.contains(lang, na=False)]['ConvertedSalary'].median()
    doesnt = df_clean[~df_clean['LanguageWorkedWith'].str.contains(lang, na=False)]['ConvertedSalary'].median()
    premium = knows - doesnt
    lang_premiums.append({'Language': lang, 'Median_With': knows, 'Median_Without': doesnt, 'Premium': premium})

premium_df = pd.DataFrame(lang_premiums).sort_values('Premium', ascending=True)

fig, ax = plt.subplots(figsize=(12, 7))
colors = ['#e74c3c' if p < 0 else '#27ae60' for p in premium_df['Premium']]
ax.barh(premium_df['Language'], premium_df['Premium'], color=colors, edgecolor='black')
ax.set_title('Salary Premium by Programming Language (Knowing vs Not Knowing)',
             fontsize=14, fontweight='bold')
ax.set_xlabel('Salary Premium (USD)')
ax.axvline(x=0, color='black', linewidth=0.8)
for i, (_, row) in enumerate(premium_df.iterrows()):
    ax.text(row['Premium'] + (500 if row['Premium'] >= 0 else -500), i,
            f"${row['Premium']:+,.0f}", va='center', fontsize=9,
            ha='left' if row['Premium'] >= 0 else 'right')
plt.tight_layout()
plt.show()

print("\n📊 Revenue Optimization Insight:")
print("   • Languages like Go, Scala, and Objective-C carry a SALARY PREMIUM")
print("     → Specialized/niche skills in high-demand create pricing power")
print("   • Common languages (PHP, CSS) show lower premiums")
print("     → Commodity skills face downward wage pressure due to oversupply")

In [ ]:
# ========================
# 7c. FINAL SUMMARY — MODEL COMPARISON TABLE
# ========================
final_sil = silhouette_score(X_scaled, kmeans_final.labels_, sample_size=5000, random_state=42)

print("=" * 80)
print("FINAL MODEL SUMMARY")
print("=" * 80)
print(f"""
┌─────────────────────────────────────────────────────────────────────────┐
│ MODEL 1: K-Means Clustering (Skill Segmentation)                      │
├─────────────────────────────────────────────────────────────────────────┤
│ Number of Clusters:     {optimal_k}                                            │
│ Silhouette Score:       {final_sil:.4f}                                      │
│ Features Used:          {len(cluster_features)}                                            │
│ Business Use:           Developer market segmentation, hiring strategy │
└─────────────────────────────────────────────────────────────────────────┘

┌─────────────────────────────────────────────────────────────────────────┐
│ MODEL 2: Linear Regression (Salary Prediction)                        │
├─────────────────────────────────────────────────────────────────────────┤
│ R² Score (Train):       {train_r2:.4f}                                      │
│ R² Score (Test):        {test_r2:.4f}                                      │
│ RMSE (Test):            ${test_rmse:,.0f}                                   │
│ Features Used:          {len(feature_cols)}                                           │
│ Business Use:           Salary benchmarking, compensation planning     │
└─────────────────────────────────────────────────────────────────────────┘

Key Economic & Business Takeaways:
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
1. DEMAND-SUPPLY: Specialized roles (DevOps, ML Engineers) command salary
   premiums due to supply shortages in the labor market.

2. HUMAN CAPITAL THEORY: Professional experience is the strongest predictor
   of salary — each additional year adds significant earning potential.

3. PRICING STRATEGY: Companies can use cluster profiles to set competitive
   salary bands for different developer tiers.

4. RISK ANALYSIS: Over-reliance on common skills (JS, HTML) carries wage
   stagnation risk; diversifying into niche technologies hedges this.

5. REVENUE OPTIMIZATION: Hiring from lower-cost clusters while upskilling
   them can optimize engineering budget allocation.
""")

print("✅ Analysis Complete!")